**Setup** <br> In this part the important packages and the data are going to be imported. The data is going to be preprocessed, so we are able to work with it.

In [1]:
# importing all the necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
from sklearn.neural_network import MLPRegressor # neural network that is going to be used
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
# importing the data and splitting it into a train and test set to work with
# important!! use index_col=0 to read df otherwise pca starts at col 58
ped = pd.read_csv("data_clean_with_dummies.csv", index_col=0)

# beginning of the code, so everyone has the same data, for reproduceability
np.random.seed(1) 

# Define a new X with the squared feature k = 0.99, 57+46
X = np.array(ped[ped.columns[57:103]])

# Output to predict
y = ped["pedestrians count"]

# Split the dataset into train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.3, random_state=72)

**Hyperparameter tuning** <br> The different hyperparameters of the used function for our neural netwwork (MLPRegressor by sklearn) are being optimized. <br> 1. Establish a base scenario with the standard parameters to compare afterwards. <br> 2. Analyze the influence of the different parameters of MLPRegressor if every other parameter is in default. <br> 3. Trying to find the best parameters using RandomizedSearchCV and GridSearchCV using subsamples of the data.

*1. Establish Base Scenario*

In [3]:
# initialize a neural network with the default parameters 
nnet_base = MLPRegressor()

In [4]:
# scale the data to work so it is less sensitive to feature scaling 
scaler = StandardScaler()
# scale the inputs
scaler.fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
# apply the same transformation to the test data to have meaningful results
scaler.fit(Xtest)
Xtest_scaled = scaler.transform(Xtest)
# scale the y different because we can't use standardscaler to a 1d array
mu, sigma = y.mean(), y.std()
ytest_scaled = (ytest - mu) / sigma
ytrain_scaled = (ytrain - mu) /sigma

In [5]:
# Fit the network to the scaled train data
nnet_base.fit(Xtrain_scaled, ytrain_scaled)

# Make predictions
ypred_train_scaled = nnet_base.predict(Xtrain_scaled)
ypred_scaled = nnet_base.predict(Xtest_scaled)

# Reconstruct outputs and scale back predictions
ytest = ytest_scaled * sigma + mu
ytrain = ytrain_scaled * sigma + mu
ypred_train = ypred_train_scaled * sigma + mu
ypred = ypred_scaled * sigma + mu
# the network predicts negative values because it is not possible to put an activation function on the output layer
# MLPRegressor uses the activation function only on the hidden layers
# Negative Values make no sense in our case, therefore we fix it and tell the network that all negative values
# should equal to zero
ypred[ypred < 0] = 0

# Compute the MAE
mae_train = mean_absolute_error(ytrain, ypred_train)
mae_test = mean_absolute_error(ytest, ypred)
# Comute R^2
r2_train = r2_score(ytrain, ypred_train)
r2_test = r2_score(ytest, ypred)

# creating a dataframe to safe all results of mae and r2
comparison = pd.DataFrame(columns=["model", "mae_train", "mae_test", "r2_train", "r2_test"])
# adding the calculated reults to the dataframe
row = pd.DataFrame(data=[["nnet_base",  mae_train, mae_test, r2_train, r2_test]],
columns=["model", "mae_train", "mae_test", "r2_train", "r2_test"])
comparison = pd.concat([comparison, row])
# merging results into dataframe and safe it as csv to add things later
comparison = comparison.reset_index().drop(columns="index")
comparison.to_csv("nnet_comparison.csv", sep=";", index=False)
comparison


,model,mae_train,mae_test,r2_train,r2_test
0,nnet_base,347.447001,456.57609,0.964814,0.942335


*2. Analyze the Influence of the Different Parameters*

In [ ]:
# conduncting the parameter analysis on the most used parameters for optimization
# how many hidden layers with how many neurons are we going to use
# 100 is the default value, other values chosen randomly to see the effect of more layers
hidden_layer_sizes = [(100,), (32, 64, 32), (64, 128, 64), (32, 64, 128, 64, 32)]
# activation function that is going to be used on the hidden layers, relu is the default value
activations = ["relu", "logistic", "tanh", "identity"]
# the solver which optimizes the weights, adam is the default value
solvers = ["sgd", "adam"]
# regularization: avoids overfitting, 0.0001 is the default value
alphas = np.linspace(0.00001, 0.001, 50)
# how high is the initial learing rate, 0.001 is the default value
learing_rate_inits = np.linspace(0.001, 0.01, 50)
# number of epochs, 200 is the default value
# often it doesn't converge then so we start at 1000
max_iters = range(1000, 2000, 20)

In [ ]:
# creating a dataframe to save the results
results = pd.DataFrame(columns=["Parameter", "ParameterValue", "MaeTrain", "MaeTest", "R2Train", "R2Test"])

# defining a function to do the analysis on the different parameters
def test_parameter(param, param_values, initialize_model):
    global results
    for param_value in param_values:
        nnet_param = initialize_model(param_value)
        # Fit the network to the train data
        nnet_param.fit(Xtrain_scaled, ytrain_scaled)
        # Make predictions
        ypred_param_train_scaled = nnet_param.predict(Xtrain_scaled)
        ypred_param_scaled = nnet_param.predict(Xtest_scaled)

        # Reconstruct outputs and scale predictions
        ypred_param_train = ypred_param_train_scaled * sigma + mu
        ypred_param = ypred_param_scaled * sigma + mu
        ypred_param[ypred_param < 0] = 0 # all neg values equal zero to make sense
        
        # Compute the MAE
        mae_param_train = mean_absolute_error(ytrain, ypred_param_train)
        mae_param_test = mean_absolute_error(ytest, ypred_param)
        # Comute R^2
        r2_param_train = r2_score(ytrain, ypred_param_train)
        r2_param_test = r2_score(ytest, ypred_param)

        # adding the calculated reults to the dataframe
        row = pd.DataFrame(data=[[param, param_value, mae_param_train, mae_param_test, r2_param_train, r2_param_test]],
        columns=["Parameter", "ParameterValue", "MaeTrain", "MaeTest", "R2Train", "R2Test"])
        results = pd.concat([results, row])

# apply the function to the different parameters
test_parameter("hidden_layer_size", hidden_layer_sizes, lambda value: MLPRegressor(hidden_layer_sizes=value, random_state=72))
test_parameter("activation", activations, lambda value: MLPRegressor(activation=value, random_state=72))
test_parameter("solver", solvers, lambda value: MLPRegressor(solver=value, random_state=72))
test_parameter("alpha", alphas, lambda value: MLPRegressor(alpha=value, random_state=72))
test_parameter("learning_rate_init", learing_rate_inits, lambda value: MLPRegressor(learning_rate_init=value, random_state=72))
test_parameter("max_iter", max_iters, lambda value: MLPRegressor(max_iter=value, random_state=72))

# merging results into dataframe and safe it as csv to work with it later on
results = results.reset_index().drop(columns="index")
results.to_csv("param_testing_nnet.csv", sep=";", index=False)

In [6]:
# reading the created dataframe and have a look at it to plot it later for the analysis
param_testing = pd.read_csv("param_testing_nnet.csv", sep=";")
param_testing

,Parameter,ParameterValue,MaeTrain,MaeTest,R2Train,R2Test
0,hidden_layer_size,"(100,)",347.696832,437.135880,0.964892,0.943203
1,hidden_layer_size,"(32, 64, 32)",297.319744,427.635196,0.972019,0.931630
2,hidden_layer_size,"(64, 128, 64)",297.511867,427.921575,0.973874,0.927907
3,hidden_layer_size,"(32, 64, 128, 64, 32)",244.434975,442.420122,0.979625,0.921200
4,activation,relu,347.696832,437.135880,0.964892,0.943203
...,...,...,...,...,...,...
155,max_iter,1900,347.696832,437.135880,0.964892,0.943203
156,max_iter,1920,347.696832,437.135880,0.964892,0.943203
157,max_iter,1940,347.696832,437.135880,0.964892,0.943203
158,max_iter,1960,347.696832,437.135880,0.964892,0.943203


In [ ]:
# defining a function for barplots for some of the parameters to compare mae and r2
def plot_bars(data_filter):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 8))
    plot_data = param_testing[param_testing["Parameter"] == data_filter]

    mae_train_axes = axes[0, 0]
    sns.barplot(data=plot_data, x="ParameterValue", y="MaeTrain", ax=mae_train_axes, palette="rocket")
    mae_train_axes.set_ylim(200, 600)

    mae_test_axes = axes[0, 1]
    sns.barplot(data=plot_data, x="ParameterValue", y="MaeTest", ax=mae_test_axes, palette="rocket")
    mae_test_axes.set_ylim(200, 600)

    r2_train_axes = axes[1, 0]
    sns.barplot(data=plot_data, x="ParameterValue", y="R2Train", ax=r2_train_axes, palette="Blues")
    r2_train_axes.set_ylim(0.9, 1)

    r2_test_axes = axes[1, 1]
    sns.barplot(data=plot_data, x="ParameterValue", y="R2Test", ax=r2_test_axes, palette="Blues")
    r2_test_axes.set_ylim(0.9, 1)

# apply the function to wanted parameters for comparison
plot_bars("hidden_layer_size")
plot_bars("activation")
plot_bars("solver")

*Interpretation:* <br> - Hidden Layers: It can be seen that "(100,)" or "(32, 64, 32)" are the best parameters while analyzing MAE and R^2 <br> - Activation: Clearly "Relu" or "Logistic" should be used as an activation function. <br> - Solver: "Adam" is the better solver on the training an the testing data.

In [ ]:
# defining a function for lineplots for some of the parameters to compare mae and r2
def plot_lines(data_filter):
    fig, axes = plt.subplots(ncols=2, figsize=(16, 8))
    plot_data = param_testing[param_testing["Parameter"] == data_filter]
    plot_data = pd.melt(plot_data, id_vars=["Parameter", "ParameterValue"], var_name="Measure", 
    value_vars=["MaeTrain", "MaeTest", "R2Train", "R2Test"], value_name="MeasuredValue")

    mae_data = plot_data[plot_data["Measure"].isin(["MaeTrain", "MaeTest"])]
    mae_axes = axes[0]
    sns.lineplot(data=mae_data, x="ParameterValue", y="MeasuredValue", hue="Measure", ax=mae_axes, palette="rocket")
    mae_axes.set_xticks([0, 24, 49])

    r2_data = plot_data[plot_data["Measure"].isin(["R2Train", "R2Test"])]
    r2_axes = axes[1]
    sns.lineplot(data=r2_data, x="ParameterValue", y="MeasuredValue", hue="Measure", ax=r2_axes, palette="Blues")
    r2_axes.set_xticks([0, 24, 49])

# apply the function to wanted parameters for comparison
plot_lines("alpha")
plot_lines("learning_rate_init")
plot_lines("max_iter")

*Interpretation:* <br> - Alpha: It should be somewhere in the middle of 0.00001 and 0.001, so the default value of 0.0001 doesn't seem to be too bad <br> - Learning Rate: A good learning rate seems to be around 0.001, like the default value. Maybe we should event try something smaller <br> - Iterations: There is no different in using 1000 or 2000 iterations. We try it nevertheless to find out if it is influenced by other parameters.

*3. Trying to optimize the Hyperparameters of the neural network using the findings above*
1. The optimal tuning of the hyperparameters would normally be done with the whole dataset. This would take for this project far too long to calculate. Therefore the data is subsetted and 10% of the data is used for the hyperparameter tuning. 
2. First using RandomizedSearchCV on the subset to find a good starting point for each parameter.
3. With the results use GridSearchCV and conduct an analysis in a range around the found parameters.

In [7]:
# subsetting the data for the optimization of the hyperparameters
ped_subset = ped.sample(frac=0.1, random_state=1)

# Define a new X of the subset with the squared feature k = 0.99, 57+46
X_subset = np.array(ped_subset[ped_subset.columns[57:103]])

# Subset of the output to predict
y_subset = ped_subset["pedestrians count"]

# Split the subset into train and test sets
Xtrain_subset, Xtest_subset, ytrain_subset, ytest_subset = train_test_split(
    X_subset, y_subset, test_size=0.3, random_state=72)

In [8]:
# scale the data of the subset, so it is less sensitive to feature scaling 
# scale the inputs
scaler.fit(Xtrain_subset)
Xtrain_scaled_subset = scaler.transform(Xtrain_subset)
# apply the same transformation to the test data to have meaningful results
scaler.fit(Xtest_subset)
Xtest_scaled_subset = scaler.transform(Xtest_subset)
# scale the y different because we can't use standardscaler to a 1d array
mu_subset, sigma_subset = y_subset.mean(), y_subset.std()
ytest_scaled_subset = (ytest_subset - mu_subset) / sigma_subset
ytrain_scaled_subset = (ytrain_subset - mu_subset) /sigma_subset

In [9]:
# defining the model and the parameters used in RandomizedSearchCV
nnet_randomsearch = MLPRegressor(random_state=72)

parameters_randomsearch = {
    "hidden_layer_sizes": [(50,), (100,), (150,), (9, 18, 6), (32, 64, 32), (12, 24, 12)],
    "activation": ["logistic", "relu"],
    "solver": ["adam"],
    "alpha": np.linspace(0.00005, 0.0005, 20),
    "learning_rate_init" : np.linspace(0.0005, 0.001, 20),
    "max_iter" : range(1000, 2000, 100)
}

In [ ]:
# running the RandomizedSearchCV
nnet_rndm_src = RandomizedSearchCV(estimator=nnet_randomsearch, param_distributions=parameters_randomsearch, cv=2)
nnet_rndm_src.fit(Xtrain_subset, ytrain_subset)
nnet_rndm_src.best_params_

*results:* <br> {'solver': 'adam', <br>
 'max_iter': 1400, <br>
 'learning_rate_init': 0.0006052631578947369, <br>
 'hidden_layer_sizes': (9, 18, 6), <br>
 'alpha': 0.00047631578947368423, <br>
 'activation': 'relu'}


In [10]:
# creating a neural network with those results and look at mae and r2
nnet_random = MLPRegressor(hidden_layer_sizes=(9, 18, 6), activation="relu", solver="adam", alpha=0.00047631578947368423,
    learning_rate_init=0.0006052631578947369, max_iter=1400, random_state=72)

# Fit the network to the scaled train data
nnet_base.fit(Xtrain_scaled, ytrain_scaled)

# Make predictions
ypred_train_scaled_random = nnet_base.predict(Xtrain_scaled)
ypred_scaled_random = nnet_base.predict(Xtest_scaled)

# Reconstruct outputs and scale back predictions
ypred_train_random = ypred_train_scaled_random * sigma + mu
ypred_random = ypred_scaled_random * sigma + mu
# the network predicts negative values because it is not possible to put an activation function on the output layer
# MLPRegressor uses the activation function only on the hidden layers
# Negative Values make no sense in our case, therefore we fix it and tell the network that all negative values
# should equal to zero
ypred_random[ypred_random < 0] = 0

# Compute the MAE
mae_train_random = mean_absolute_error(ytrain, ypred_train_random)
mae_test_random = mean_absolute_error(ytest, ypred_random)
# Comute R^2
r2_train_random = r2_score(ytrain, ypred_train_random)
r2_test_random = r2_score(ytest, ypred_random)

# adding the calculated reults to the dataframe
row = pd.DataFrame(data=[["nnet_randomsearch",  mae_train_random, mae_test_random, r2_train_random, r2_test_random]],
columns=["model", "mae_train", "mae_test", "r2_train", "r2_test"])
comparison = pd.concat([comparison, row])
# merging results into dataframe and safe it as csv to add things later
comparison = comparison.reset_index().drop(columns="index")
comparison.to_csv("nnet_comparison.csv", sep=";", index=False)
comparison

,model,mae_train,mae_test,r2_train,r2_test
0,nnet_base,347.447001,456.576090,0.964814,0.942335
1,nnet_randomsearch,349.732476,457.666502,0.964785,0.941682


In [ ]:
# scale the data so it is less sensitive to feature scaling
scaler = StandardScaler()
# scale the inputs
scaler.fit(X)
X = scaler.transform(X)

# scale the y different because we can't use standardscaler to a 1d array
mu, sigma = y.mean(), y.std() # We will use this to scale back to original values!
y = (y - mu) / sigma

scorers = ["r2", "neg_mean_absolute_error"]

parameters= {
    "hidden_layer_sizes": [(32, 64, 32), (64, 128, 64), (32, 64, 128, 64, 32)],
    "activation": ["tanh", "relu"],
    "solver": ["sgd", "adam"],
    "alpha": [0.0001, 0.0005, 0.00005],
    "learning_rate_init" : [0.001, 0.01],
    "max_iter" : [1000, 2000]
}

model1 = GridSearchCV(MLPRegressor(), parameters, cv=5, scoring=scorers, refit=False)
model1.fit(X, y)

results = pd.DataFrame()
params = model1.cv_results_["params"]

maes = model1.cv_results_["mean_test_neg_mean_absolute_error"]
r2s = model1.cv_results_["mean_test_r2"]

results["Params"] = pd.Series(params)
results["MeanMAE"] = pd.Series(maes)
results["MeanR2"] = pd.Series(r2s)

results.sort_values("MeanR2", inplace=True)

results.to_csv("results_nnet_gs.csv", sep=";", index=False)

results

In [ ]:
# modelling the MLPRegressor with best parameters
# importing the data and splitting it to work with
# important!! use index_col=0 to read df otherwise pca starts at col 58
ped = pd.read_csv("data_clean_with_dummies.csv", index_col=0)

# beginning of the code, so everyone has the same
np.random.seed(1) # Set the random seed for reproduceability

# Define a new X with the squared feature k = 0.99, 57+46
X = np.array(ped[ped.columns[57:103]])

# Output to predict
y = ped["pedestrians count"]

# Split the dataset into train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.3, random_state=72)

In [ ]:
# initialize a neural network with the standard parameters 
nnet_optimization = MLPRegressor(hidden_layer_sizes=(32, 64, 32), activation="relu", solver="adam",
                    alpha=0.0001, learning_rate_init=0.01, max_iter=1000)

In [ ]:
# scale the data to work so it is less sensitive to feature scaling 
scaler = StandardScaler()
# scale the inputs
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
# apply the same transformation to the test data to have meaningful results
Xtest = scaler.transform(Xtest)
# scale the y different because we can't use standardscaler to a 1d array
mu, sigma = y.mean(), y.std() # We will use this to scale back to original values!
ytest = (ytest - mu) / sigma
ytrain = (ytrain - mu) /sigma

In [ ]:
# Fit the network to the train data
nnet_optimization.fit(Xtrain, ytrain)

# Make predictions
ypred_train = nnet_optimization.predict(Xtrain)
ypred = nnet_optimization.predict(Xtest)

# Reconstruct outputs and scale predictions
ytest = ytest * sigma + mu
ytrain = ytrain * sigma + mu
ypred_train = ypred_train * sigma + mu
ypred = ypred * sigma + mu

# Compute the MAE
mae_train = mean_absolute_error(ytrain, ypred_train)
mae_test = mean_absolute_error(ytest, ypred)
# Comute R^2
r2_train = r2_score(ytrain, ypred_train)
r2_test = r2_score(ytest, ypred)

print(f"The mean absolute error of the training data is {mae_train:>10.2f}")
print(f"The R^2 of the training data is {r2_train:>10.2f}")
print(20 * "*")
print(f"The mean absolute error of the testing data is {mae_test:>10.2f}")
print(f"The R^2 of the testing data is {r2_test:>10.2f}")

*Results:* <br> The mean absolute error of the training data is 339.24 <br>
The R^2 of the training data is 0.96 <br>
******************** <br>
The mean absolute error of the testing data is 400.66 <br>
The R^2 of the testing data is 0.94

In [ ]:
# scale the data so it is less sensitive to feature scaling
scaler = StandardScaler()
# scale the inputs
scaler.fit(X)
X = scaler.transform(X)

# scale the y different because we can't use standardscaler to a 1d array
mu, sigma = y.mean(), y.std() # We will use this to scale back to original values!
y = (y - mu) / sigma

scorers = ["r2", "neg_mean_absolute_error"]

parameters= {
    "hidden_layer_sizes": [(32, 64, 32), (100,)],
    "activation": ["relu"],
    "solver": ["adam"],
    "alpha": [0.0001],
    "learning_rate_init" : [0.001],
    "batch_size" : [16, 32, 64],
    "max_iter" : [1000]
}

model1 = GridSearchCV(MLPRegressor(), parameters, cv=5, scoring=scorers, refit=False)
model1.fit(X, y)

results = pd.DataFrame()
params = model1.cv_results_["params"]

maes = model1.cv_results_["mean_test_neg_mean_absolute_error"]
r2s = model1.cv_results_["mean_test_r2"]

results["Params"] = pd.Series(params)
results["MeanMAE"] = pd.Series(maes)
results["MeanR2"] = pd.Series(r2s)

results.sort_values("MeanR2", inplace=True)

results.to_csv("results_nnet_gs2.csv", sep=";", index=False)

results